In [13]:
!pip install langchain langchain-community gradio python-dotenv
!pip install groq

In [14]:
from google.colab import userdata
api_key_WebSearch =userdata.get('WebSearch')
api_key_coder =userdata.get('Chat_with_Your_Context')

In [15]:
import os
import gradio as gr
import requests
import json
import time
from typing import Optional, List, Any, Union, Dict
from pydantic import PrivateAttr
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.llms.base import LLM
from groq import Groq

In [17]:

# --- Setup LLM using Groq ---
class GroqLLM(LLM):
    model: str = "deepseek-r1-distill-llama-70b"
    temperature: float = 0.6
    top_p: float = 0.95
    max_tokens: int = 4096
    api_key: Optional[str] = api_key_coder

    _client: Groq = PrivateAttr()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._client = Groq(api_key=self.api_key)

    @property
    def _llm_type(self) -> str:
        return "groq-llm"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self._client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.temperature,
            top_p=self.top_p,
            max_tokens=self.max_tokens,
            stop=stop,
            stream=False
        )
        return response.choices[0].message.content.strip()

# --- Activate LLM ---
llm = GroqLLM()

In [18]:
# --- Web Search Tool ---
class WebSearchError(Exception):
    """Custom exception for web search failures"""
    pass

def web_search(query: str, max_retries: int = 3, timeout: int = 10) -> str:
    """
    Performs a web search using the Tavily API and returns the most relevant result.

    Args:
        query (str): The search query string
        max_retries (int): Maximum number of retry attempts (default: 3)
        timeout (int): Request timeout in seconds (default: 10)

    Returns:
        str: The content of the most relevant search result or an error message

    Raises:
        WebSearchError: If the search fails after all retry attempts

    Example:
        >>> result = web_search("latest AI research papers 2024")
        >>> print(result)
    """
    api_key = api_key_WebSearch  # Prefer environment variable

    if not api_key:
        raise WebSearchError("API key not configured for web search")

    headers = {
        "Authorization": api_key,
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

    payload = {
        "query": query,
        "include_raw_content": True,
        "max_results": 3  # Get top 3 results for better context
    }

    last_error = None

    for attempt in range(max_retries):
        try:
            response = requests.post(
                "https://api.tavily.com/search",
                headers=headers,
                json=payload,
                timeout=timeout
            )

            response.raise_for_status()
            data = response.json()

            # Process results with multiple fallback options
            if not data.get("results"):
                return "No relevant results found."

            # Return concatenated content from top 3 results for better context
            contents = [r.get("content", "") for r in data["results"][:3] if r.get("content")]
            return "\n\n".join(contents) or "No readable content available."

        except requests.exceptions.RequestException as e:
            last_error = str(e)
            if attempt < max_retries - 1:
                time.sleep(1 * (attempt + 1))  # Exponential backoff
            continue

    raise WebSearchError(f"Web search failed after {max_retries} attempts. Last error: {last_error}")

WebSearchTool = Tool.from_function(
    func=web_search,
    name="WebSearch",
    description="""
    Powerful web search capability that:
    - Retrieves the most up-to-date information from the web
    - Returns summarized content from multiple sources
    - Handles complex queries with multiple retries
    Use when you need current information beyond your knowledge cutoff.
    """,
    handle_tool_error=True
)

In [19]:
# --- Context Presence Judge Tool ---
def build_context_presence_tool(llm: Any, prompt_path: str) -> Tool:
    """
    Creates a tool that evaluates whether a user query contains sufficient context.

    This tool:
    - Loads a prompt template from a file
    - Formats the template with user input
    - Uses an LLM to judge context adequacy
    - Returns a ready-to-use Tool object

    Args:
        llm (Any): The language model instance to use for evaluation
        prompt_path (str): Path to the prompt template file

    Returns:
        Tool: Configured tool for context presence evaluation

    Raises:
        FileNotFoundError: If the prompt template file doesn't exist
        ValueError: If the prompt template is malformed

    Example:
        >>> judge_tool = build_context_presence_tool(llm, "prompts/context_check.txt")
        >>> judge_tool.run("What's the capital?||Speaking about France")
    """
    try:
        # Load prompt template with explicit encoding and error checking
        with open(prompt_path, "r", encoding="utf-8") as file:
            prompt_template = file.read().strip()

            if not prompt_template:
                raise ValueError("Prompt template file is empty")

            if "{input}" not in prompt_template:
                raise ValueError("Prompt template must contain '{input}' placeholder")

        def judge_context_presence(input_text: str) -> str:
            """
            Evaluates whether the input contains sufficient context.

            Args:
                input_text (str): User input to evaluate

            Returns:
                str: LLM's judgment about context adequacy
            """
            try:
                formatted_prompt = prompt_template.replace("{input}", input_text)
                response = llm(formatted_prompt)
                return response.strip()
            except Exception as e:
                return f"Error evaluating context: {str(e)}"

        return Tool.from_function(
            func=judge_context_presence,
            name="ContextPresenceJudge",
            description="""Evaluates whether user queries contain sufficient context.
            Use this when you need to determine if additional context is required
            to properly answer a question.""",
            handle_tool_error=True
        )

    except FileNotFoundError:
        raise FileNotFoundError(f"Prompt template file not found at {prompt_path}")
    except Exception as e:
        raise RuntimeError(f"Failed to create context judge tool: {str(e)}")

In [20]:
# --- Enhanced Context Processing Tool ---
def Context_Relevance_Splitter(input_text: str) -> Union[str, Dict[str, str]]:
    """
    Advanced context processing with improved error handling and answer extraction.
    """
    try:
        # If there is no context (no ||)
        if "||" not in input_text:
            return {
                "core_question": input_text.strip(),
                "background": "",
                "answer_found": False,
                "query_for_search": input_text.strip()
            }

        # Split context from question
        parts = [p.strip() for p in input_text.split("||", 1) if p.strip()]

        # Check if there are two parts (context and question)
        if len(parts) < 2:
            return "⚠ Please use the format: question||context"

        context, question = parts[0], parts[1]

        # Check context relevance
        relevance_prompt = f"""
        Is the following context relevant to the question?
        Context: {context}
        Question: {question}

        Answer only with yes or no:
        """

        relevance_response = llm(relevance_prompt).strip().lower()

        if "yes" not in relevance_response and "نعم" not in relevance_response:
            return {
                "background": context,
                "core_question": question,
                "answer_found": False,
                "query_for_search": question,
                "message": "🚫 Irrelevant context - will search for answer externally"
            }

        # Try to extract answer from context
        answer_extraction_prompt = f"""
        Based on the following context, answer the question.
        If the answer is not found in the context, say "ANSWER_NOT_FOUND".

        Context: {context}
        Question: {question}

        Answer:
        """

        answer_response = llm(answer_extraction_prompt).strip()

        # Check if answer was found in context
        if "ANSWER_NOT_FOUND" in answer_response.upper():
            return {
                "background": context,
                "core_question": question,
                "answer_found": False,
                "query_for_search": question,
                "extracted_answer": None,
                "message": "Answer not found in context - requires external search"
            }
        else:
            return {
                "background": context,
                "core_question": question,
                "answer_found": True,
                "query_for_search": None,
                "extracted_answer": answer_response,
                "message": "Answer successfully extracted from context"
            }

    except Exception as e:
        # In case of any unexpected error
        return {
            "core_question": input_text.strip(),
            "background": "",
            "answer_found": False,
            "query_for_search": input_text.strip(),
            "error": str(e)
        }

# --- Agent Tool Definition ---
Context_Relevance_Splitter_tool = Tool.from_function(
    func=Context_Relevance_Splitter,
    name="ContextProcessor",
    description="""
    Advanced context processing and answer extraction tool:
    1. Checks relevance of context to the question
    2. Attempts to extract answer from context
    3. If answer not found, returns query for external search
    4. Splits context from the core question
    Use format: question||context
    Returns: answer_found, extracted_answer, query_for_search
    """
)

In [21]:
# Make sure this prompt file exists with appropriate content
context_tool = build_context_presence_tool(llm, prompt_path="context_judge_prompt.txt")

# --- Agent initialization with all tools ---
tools = [context_tool,
    Context_Relevance_Splitter_tool,
    WebSearchTool
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True,
    max_iterations=5
)

# --- Enhanced Agent Runner Function ---
def run_agent(user_input):
    try:
        # Initial context processing
        context_result = Context_Relevance_Splitter_tool(user_input)

        # If result is a string (error message)
        if isinstance(context_result, str):
            if "🚫" in context_result or "⚠" in context_result:
                return context_result
            else:
                final_question = user_input
                background = ""
        else:
            # If result is a dictionary (successful processing)
            final_question = context_result.get('core_question', user_input)
            background = context_result.get('background', '')

        # Build final agent input
        if background:
            enhanced_input = f"""
            Context Background: {background}
            Question: {final_question}
            """
        else:
            enhanced_input = final_question

        # Run agent with final input
        return agent.run(enhanced_input)

    except Exception as e:
        return f"⛔ Unexpected error occurred: {str(e)}\nPlease rephrase your question or try again later."

# --- Enhanced Gradio Interface ---
interface = gr.Interface(
    fn=run_agent,
    inputs=gr.Textbox(
        lines=3,
        placeholder="Enter your question here...\nTo add context use format: Question||Context\nExample: What's France's capital||Speaking about a European country"
    ),
    outputs="text",
    title="🤖 Context-Aware Smart Agent",
    description="""
    Advanced system for understanding complex questions:
    - Supports external context using ||
    - Automatically get the answer from context
    - Answers directly or searches when needed
    """,
    examples=[
        ["العاصمة السعودية هي الرياض||ما هي عاصمة مصر؟"],
        ["العاصمة السعودية هي الرياض||ما هي عاصمة السعودية؟"]
    ]
)

# --- Launch Interface ---
if __name__ == "__main__":
    interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://63fca5194ca9128f67.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
